# 2.5 自动微分

推荐一个最近刷到的讲自动微分的宝藏视频，来自`ZOMI酱`的自动微分系列课程：https://space.bilibili.com/517221395/channel/collectiondetail?sid=758156&ctype=0

`MindSpore` 自动微分官方教程：https://www.mindspore.cn/tutorials/zh-CN/r1.7/beginner/autograd.html

### 2.5.1 一个简单的例子

假设我们想对函数 $y = 2x^Tx$ 关于列向量 $x$ 求导。 首先，我们创建变量 $x$ 并为其分配一个初始值。

In [2]:
import mindspore.numpy as mnp

x = mnp.arange(4.0)
x

Tensor(shape=[4], dtype=Float32, value= [0.00000000e+000, 1.00000000e+000, 2.00000000e+000, 3.00000000e+000])

MindSpore 自动微分的实现方式：

MindSpore现有版本同时支持静态图和动态图，即GRAPH_MODE和PYNATIVE_MODE，为了将二者统一，整体进行自动微分的方式为静态图方式，即：

将函数视为一个完整的计算图，先进行编译，后执行。因此不像Pytorch一样将梯度grad直接绑定在Tensor上，而是整图运算后，再通过取梯度的算子进行梯度的提取。（动态图模式也并非纯Python执行，二者均采取静态图的策略）因此，和Pytorch有如下差异：

1、MindSpore采用函数式编程，因此所有计算公式表达都采用函数进行表示；

2、需要通过 `ops.GradOperation` 算子来获取梯度

`ops.GradOperation` 属性如下：

`get_all`：计算梯度，如果等于False，获得第一个输入的梯度，如果等于True，获得所有输入的梯度。默认值为False。

`get_by_list`：是否对权重参数进行求导，默认值为False。

`sens_param`：是否对网络的输出值做缩放以改变最终梯度，默认值为False。

现在我们计算 $y$，由于 MindSpore 采用函数式编程，所以 $y$ 需要显式注册为 function。

In [3]:
def forward(x):
    return 2 * mnp.dot(x, x)

y = forward(x)
y

Tensor(shape=[], dtype=Float32, value= 28)

通过调用 `ops.GradOperation` 算子来自动计算 $y$ 关于 $x$ 每个分量的梯度。

In [4]:
import mindspore.ops as ops

grad_op = ops.GradOperation(get_all=True)
gradient_function = grad_op(forward)
x_grad = gradient_function(x)[0]
x_grad

Tensor(shape=[4], dtype=Float32, value= [0.00000000e+000, 4.00000000e+000, 8.00000000e+000, 1.20000000e+001])

In [5]:
x_grad == 4 * x

Tensor(shape=[4], dtype=Bool, value= [ True,  True,  True,  True])

现在让我们计算 $x$ 的另一个函数。

In [6]:
def forward(x):
    return x.sum()

x_grad = grad_op(forward)(x)[0]
x_grad

Tensor(shape=[4], dtype=Float32, value= [1.00000000e+000, 1.00000000e+000, 1.00000000e+000, 1.00000000e+000])

### 2.5.2 非标量变量的反向传播

当 $y$ 不是标量时，向量 $y$ 关于向量 $x$ 的导数的最自然解释是一个矩阵。 对于高阶和高维的 $y$ 和 $x$，求导的结果可以是一个高阶张量。

然而，虽然这些更奇特的对象确实出现在高级机器学习中（包括深度学习中）， 但当我们调用向量的反向计算时，我们通常会试图计算一批训练样本中每个组成部分的损失函数的导数。 这里，我们的目的不是计算微分矩阵，而是单独计算批量中每个样本的偏导数之和。

In [7]:
def forward(x):
    y = x * x
    return y.sum()

x_grad = grad_op(forward)(x)[0]
x_grad

Tensor(shape=[4], dtype=Float32, value= [0.00000000e+000, 2.00000000e+000, 4.00000000e+000, 6.00000000e+000])

### 2.5.3 分离计算

有时，我们希望将某些计算移动到记录的计算图之外。

下面的反向传播函数计算 $z=u*x$ 关于 $x$ 的偏导数，同时将 $u$ 作为常数处理， 而不是 $z=x*x*x$ 关于 $x$ 的偏导数。

In [8]:
def forward(x):
    y = x * x
    u = ops.stop_gradient(y)
    z = u * x
    return z.sum(), u

z, u = forward(x)
x_grad = grad_op(forward)(x)[0]
x_grad == u

Tensor(shape=[4], dtype=Bool, value= [ True,  True,  True,  True])

In [9]:
def forward(x):
    y = x * x
    u = ops.stop_gradient(y)
    z = u * x
    return y.sum()

y = forward(x)
y_grad = grad_op(forward)(x)[0]
y_grad == 2 * x

Tensor(shape=[4], dtype=Bool, value= [ True,  True,  True,  True])

### 2.5.4 Python控制流的梯度计算

使用自动微分的一个好处是： 即使构建函数的计算图需要通过 Python 控制流（例如，条件、循环或任意函数调用），我们仍然可以计算得到的变量的梯度。 在下面的代码中，while 循环的迭代次数和 if 语句的结果都取决于输入 a 的值。

In [10]:
from mindspore import context
context.set_context(mode=context.PYNATIVE_MODE)

def forward(a):
    b = a * 2
    while mnp.norm(b) < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

a = mnp.randn()
d = forward(a)

a_grad = grad_op(forward)(a)[0]
a_grad == d / a

Tensor(shape=[1], dtype=Bool, value= [ True])